# Fully entangled PQC using Amazon Braket

Very similar example [here](https://github.com/amazon-braket/amazon-braket-examples/tree/main/examples/hybrid_quantum_algorithms/QAOA)
and code adapts from it.

In [ ]:
# general imports
import matplotlib.pyplot as plt

# magic word for producing visualizations in notebook
%matplotlib widget

from braket.circuits import Circuit
from braket.circuits import FreeParameter
from braket.devices import LocalSimulator
from braket.circuits.observables import Z

import numpy as np

In [ ]:
# Ensure consistent results
np.random.seed(0)

# Set up device: Local Simulator
device = LocalSimulator()

In [ ]:
def rotations(wire, params):
    circuit = Circuit()
    circuit.rz(wire, params[0])
    circuit.ry(wire, params[1])
    circuit.rz(wire, params[2])
    return circuit

def entangle(n_qubits):
    circuit = Circuit()
    if n_qubits <= 0:
        return circuit
    for ii in range(n_qubits):
        circuit.cnot(ii, (ii + 1) % n_qubits)
    return circuit

def training_layer(n_qubits, params):
    circuit = Circuit()
    for ii in range(n_qubits):
        circuit.add(rotations(ii, params[ii,:]))
    circuit.add(entangle(n_qubits))
    return circuit

def encoding_layer(n_qubits, params):
    circuit = Circuit()
    for ii in range(n_qubits):
        circuit.rx(ii, params[ii])
    return circuit

In [ ]:
n_qubits = 3
layers = 2

In [ ]:
def enc_params(n_qubits, layers):
    params = np.empty((layers, n_qubits), object)
    for ll in range(layers):
        for nn in range(n_qubits):
            name = f"enc_l_{ll}_q_{nn}"
            params[ll, nn] = FreeParameter(name)
    return params

def rot_params(n_qubits, layers):
    params = np.empty((layers, n_qubits, 3), object)
    for ll in range(layers):
        for nn in range(n_qubits):
            for aa in range(3):
                name = f"rot_l_{ll}_q_{nn}_a_{aa}"
                params[ll, nn, aa] = FreeParameter(name)
    return params

In [ ]:
def circuit(n_qubits, layers):
    circuit = Circuit()
    free_enc_params = enc_params(n_qubits, layers)
    free_rot_params = rot_params(n_qubits, layers)
    for ii in range(layers):
        circuit.add(encoding_layer(n_qubits, free_enc_params[ii]))
        circuit.add(training_layer(n_qubits, free_rot_params[ii]))
    return circuit

In [ ]:
print(circuit(n_qubits, layers))

In [ ]:
def init_params(n_qubits, layers):
    params = {}
    for ll in range(layers):
        for nn in range(n_qubits):
            name = f"enc_l_{ll}_q_{nn}"
            params[name] = np.random.uniform(0, 2*np.pi)
            for aa in range(3):
                name = f"rot_l_{ll}_q_{nn}_a_{aa}"
                params[name] = np.random.uniform(0, 2*np.pi)
    return params

In [ ]:
device.run(circuit(n_qubits, layers), shots = 1000, inputs = init_params(n_qubits, layers)).result()

In [ ]:
def expectation(probs):
    # use expectation value to predict efficiency
    val = 0
    for key, value in probs.items():
        eig = 1
        for char in key:
            if char == "1":
                eig *= -1
        val += eig * value
    return val

def map_expval(expval, start, stop):
    return start + (stop - start)/2 * (expval + 1)

In [ ]:
def classifier(probs, map_dict):
    # use global classifer to predict efficiency
    res = {}
    for key, value in probs.items():
        res.append(map_dict[key], value)
    return res

In [ ]:
def objective_function(circuit, params_dict, device, shots, tracker, verbose):
    if verbose:
        print("=" * 80)
        print("Calling the quantum circuit. Cycle:", tracker["count"])

    # classically simulate the circuit
    # set the parameter values using the inputs argument
    # execute the correct device.run call depending on whether the backend is local or cloud based
    if isinstance(device, LocalSimulator):
        task = device.run(circuit, inputs=params_dict, shots = shots)
    else:
        task = device.run(
            circuit,
            shots=n_shots,
            inputs=params_dict,
            poll_timeout_seconds=3 * 24 * 60 * 60,
        )

    # get result for this task
    result = task.result()
    print(result.values)
    measurement_probabilities = result.measurement_probabilities

    expval = expectation(measurement_probabilities)
    eff = map_expval(expval, 0, 1)

    if verbose:
        print("Expectation value:", expval)
        print("Mapped efficiency:", eff)

    # update tracker
    tracker.update({"count": tracker["count"] + 1, "res": result})
    tracker["expval"].append(expval)
    tracker["params"].append(params_dict)

    return eff

In [ ]:
# set tracker to keep track of results
tracker = {
    "count": 1,  # Elapsed optimization steps
    "expval": [],  # Expectation at each step
    "res": None,  # Quantum result object
    "params": [],  # Track parameters
}

circ = circuit(n_qubits, layers)
circ.expectation(Z(0) @ Z(1) @ Z(2))
objective_function(circ, init_params(n_qubits, layers), device, 100, tracker, True) 